# Bitcoin cycles

Bitcoin prices in relation to halving cycles.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from btc_cycles import Bitcoin

load_dotenv()

ModuleNotFoundError: No module named 'pandas'

## Bitcoin

`Bitcoin` is the main object.
Fetches data from the selected source, computes metrics (ATH and cycle progress).

*Available sources*:
- `cryptocompare`: [cryptocompare](https://www.cryptocompare.com/) requires API key
- `coinmarketcap`: [coinmarketcap](https://coinmarketcap.com/) requires API key
- `coinmarketcap-free`: [coinmarketcap](https://coinmarketcap.com/) **broken legacy API**, still available hoping for a fix ([see here](https://github.com/guptarohit/cryptoCMD/issues/86)).

*Arguments*
- `currency` (str) : currency. Defaults to USD.
- `source` (str) : source of price data
- `api_key` (str) : API key to selected source

*Attributes*
- `prices` (pd.DataFrame) : pandas dataframe of prices
- `halvings` (pd.DataFrame) : pandas dataframe of halvings data
- `predicted_halving_block` (int): block of next halving
- `predicted_halving_date` (datetime): predicted date of next halving 

In [ ]:
btc = Bitcoin(
    source="cryptocompare",  # "coinmarketcap", "cryptocompare"
    api_key=os.getenv("CRYPTOCOMPARE_API_KEY"),  # only for cryptocompare, coinmarketcap
)

: 

In [ ]:
print(
    f"Next halving (block {btc.predicted_halving_block}) is predicted on {btc.predicted_halving_date}"
)

: 

In [ ]:
btc.halvings.head(10)

: 

In [ ]:
btc.prices

: 

## Plots

The method `plot` returns a `matplotlib.Figure` radial chart of the prices.

Arguments:
- `from_date` (str or datetime): date from which data is plotted. Some sources offers unlimited historical data and some filtering might be needed.
- `theme` (str or dict): theme of the plot. Default is `light`, or `dark` for dark mode. Custom themes can be passed as a dictionary. Colors can also be customized singularly.

In [ ]:
fig = btc.plot(
    from_date="2012-11-28",  # start date
    # theme="dark",  # dark or light, or dict with custom colors.
    # theme = {
    #     "now_line": "red"
    # },
)
# # fig is a matplotlib figure object
fig.savefig("bitcoin.png")

: 

: 